In [1]:
import os
OPENAI_API_BASE= os.environ.get("OPENAI_API_BASE", "https://api-ai.numerique-interieur.com/v1") 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(api_key=OPENAI_API_KEY, openai_api_base=OPENAI_API_BASE, temperature=0)

/home/jerome/miniforge3/envs/dev/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
text = ""
with open("../big_text.txt", 'r') as f:
    text = f.read()

In [14]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(text)

prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in French"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

/home/jerome/miniforge3/envs/dev/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [16]:
print(result['output_text'])

Le lendemain matin, Ned Land et le narrateur découvrirent l'île de Tarawa, située à deux milles au large de la côte de Papouasie-Nouvelle-Guinée. Ils trouvèrent des arbres, des animaux terrestres et des porteurs de couteaux. Le capitaine Nemo expliqua qu'il voulait suivre les traces du _Nautilus_ et découvrir l'endroit où il avait échoué.

Ils naviguèrent pendant plusieurs jours, passant devant d'autres îles peuplées de Papouas. Le narrateur remarqua que les habitants étaient effrayés par la présence du monstre échoué dans leur baie. Ils se rapprochèrent alors du _Nautilus_ et l'examinaient avec attention.

Le capitaine Nemo décida d'abord de s'approcher à portée de tir des habitants pour les observer de plus près. Il fit ensuite une démonstration de son arbalète, en utilisant un couteau comme flèche et en touchant le but avec précision. Les Papouas étaient fascinés par cette démonstration et commencèrent à s'approcher du _Nautilus_.

Le capitaine Nemo décida alors de faire une démonst

In [7]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter


# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [8]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [9]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [11]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(pages)

In [13]:
result_map_reduce = map_reduce_chain.run(split_docs) 

In [3]:

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../Malo_Adler_Thesis.pdf")
pages = loader.load()

In [4]:

text = ""
for page in pages:
    text += page.page_content

llm.get_num_tokens(text)

42528

In [5]:
from textrank import build_text_prompt

prompt_template = """Ecrit un résumé en français du texte suivant en détaillant le plus possible:
"{text}"
RESUME:"""
prompt = PromptTemplate.from_template(prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

text_prompt = build_text_prompt(text, 3000)

chain = load_summarize_chain(llm, chain_type="stuff")

result = llm_chain.invoke({'text': text_prompt})

In [15]:
result_map_reduce

"\nThe given text discusses various aspects related to natural language processing (NLP), machine learning, and data analysis using pre-trained language models (LLMs). The first document introduces the BERTScore metric, which uses contextual word embeddings from the BERT model to compare the semantics of two texts by comparing the embeddings of each word in one text to the embeddings of each word in the other text. The second and third documents discuss various metrics for evaluating factuality and summarization tasks using pre-trained LLMs, including SelfCheckGPT-Prompt and SelfCheckGPT-BERT, which aim to provide a more comprehensive evaluation of language models' ability to generate accurate and coherent summaries by generating multiple summaries and checking if the model's output is supported by each summary. The third document evaluates different pre-trained LLMs for abstractive summarization using basic summarization prompts, and it is observed that larger models generally perform

In [6]:
result

{'text': "Le but principal de cette thèse est d'évaluer comment les modèles de langage pré-entraînés peuvent se comporter sur ce type de données spécifiques et comment améliorer ces performances. Une méthode hybride est souvent considérée pour résoudre les contraintes de taille (voir Section 2.4.4), qui consiste à réduire une version longue du texte par des méthodes extraitives, puis à générer un résumé abstrait de cette version réduite. Cette approche présente plusieurs avantages: le résumé produit est plus proche d'un texte écrit à la main; le texte généré est plus facile à comprendre et plus lisible; le résultat peut être potentiellement plus exhaustif.\n\nL'algorithme K-Means, appliqué à la summarisation textuelle extraitives, présente un inconvénient majeur: le nombre de phrases souhaitées dans le résumé final doit être spécifié en avance car l'algorithme K-Means nécessite de connaître avanthand le nombre de clusters à calculer. Enfin, la longueur du contexte de 4096 tokens (envir

In [7]:
text_prompt

'For this reason, the other main objective of this thesis\nwill be to evaluate how pre-trained language models can perform on this specific data, and how\nthese performances can be improved.However, abstractive methods having size constraints (see Section 2.4.4), a hybrid strategy\nis often considered [5], consisting of first reducing a long text with extractive methods, and then\ngenerating an abstractive summary of this reduced version.It has several significant advantages:\n• The produced summaries are closer to a human-written text;\n• The generated text is more understandable and easier to read;\n• The result can potentially be more exhaustive.Figure 8: The K-Means algorithm, applied to extractive text summarization\nThe main disadvantage of this algorithm is that the number of sentences wanted in the final sum-\nmary must be specified in advance, because the K-Means algorithm requires knowing beforehand\nthe number of clusters to compute.Finally, the context length of 4096 tokens